In [1]:
import pandas as pd

## The extracting stage

In [7]:
raw_jobs = pd.read_pickle("data/raw_jobs_copy.pkl")
raw_jobs.head()

,url,job_info,experience_level
0,https://www.indeed.com/viewjob?jk=00a4d7932929...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Entry level
1,https://www.indeed.com/viewjob?jk=0101872c28a8...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Entry level
2,https://www.indeed.com/viewjob?jk=011d3dd5eb0c...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Senior level
3,https://www.indeed.com/viewjob?jk=011d508f79b1...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Senior level
4,https://www.indeed.com/viewjob?jk=013d358f1a88...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Mid level


## The transforming stage

In [8]:
jobs = pd.read_pickle("data/jobs.pkl")
jobs.head()

,url,job_title,company_name,company_rating,number_of_reviews,annual_salary,location,remote,full_time,temporary,internship,experience_level,degree,requirements
0,https://www.indeed.com/viewjob?jk=00a4d7932929...,Physicochemical Data Scientists,ORAU,3.6,27,63980.8,"Research Triangle Park, NC 27711",False,True,False,False,Entry level,Bachelor,"python,r,sql,programming languages,database,java"
1,https://www.indeed.com/viewjob?jk=0101872c28a8...,Jr Data Scientist (New York),Viacom,3.9,1842,None,"New York, NY 10036",False,True,False,False,Entry level,No degree,"power bi,python,r,sql,programming languages,da..."
2,https://www.indeed.com/viewjob?jk=011d3dd5eb0c...,Data Scientist - Lead,Seattle Children's,3.9,378,None,"Seattle, WA",False,True,False,False,Senior level,Bachelor,"power bi,python,r,sql,programming languages,da..."
3,https://www.indeed.com/viewjob?jk=011d508f79b1...,Senior Data Analyst/Data Analyst,None,4.2,363,66000.0,"Reno, NV",False,True,False,False,Senior level,Bachelor,"c,r,sql,programming languages,javascript,database"
4,https://www.indeed.com/viewjob?jk=013d358f1a88...,Data Analyst,None,None,None,92150,None,True,True,False,False,Mid level,No degree,"power bi,data warehousing,google cloud,sql,dat..."


In [9]:
reqs_df = pd.read_pickle("data/requirements.pkl")
reqs_df.head()

,group_of_requirements,requirement,experience_level,degree,remote,full_time,temporary,internship,number_of_occurrences
0,sql,sql,Entry level,No degree,True,True,True,True,0
1,sql,sql,Entry level,No degree,True,True,True,False,0
2,sql,sql,Entry level,No degree,True,True,False,True,0
3,sql,sql,Entry level,No degree,True,True,False,False,5
4,sql,sql,Entry level,No degree,True,False,True,True,0


## The loading stage

The loading stage is happening in "load_raw_jobs_to_db.py" and "load_features_to_db.py".

In this stage I connected to AWS RDS database using pymysql Python library, created table and loaded the raw_jobs and jobs datasets using MySQL to the database.

Below you can see the tables schemas.

## The analysis stage